In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### 1. Data loading and preprocessing

In [2]:
# Scaling the image (dividing a value of each pixels by 255, in order to get values from 0 to 1)  
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [4]:
# Loading the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', shuffle_files=True, with_info=True, as_supervised=True)

# Dividing the dataset into train and test sets
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# Choosing the size for validation set (as 10% of the train set's size)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
print('num_validation_samples:', num_validation_samples)

# Saving the size of test set into variable
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
print('num_test_samples:', num_test_samples)

# Scaling train and validation data (dividing by 255)
scaled_train_and_validation_data = mnist_train.map(scale)

# Scaling test data (dividing by 255)
scaled_test_data = mnist_test.map(scale)

# Shuffling the data
buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

# Dividing into train and validation datasets
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Batching the train data
mini_batch_size = 100
train_data = train_data.batch(mini_batch_size)

# Batching validation and test data: they don't need to be batched, 
# but the model will require all data in batch form, so we take batch size as the whole set's size   
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

# make the same shape and object properties for validation data as for the train data (itarable)
validation_inputs, validation_targets = next(iter(validation_data))

num_validation_samples: tf.Tensor(6000, shape=(), dtype=int64)
num_test_samples: tf.Tensor(10000, shape=(), dtype=int64)
